<a href="https://colab.research.google.com/github/GeorgiyAleksanyan/14-332-472-01-ROBOTICS-COMP-VISION-Classify-Dog-vs-Cat-Kaggle-dataset-with-two-different-networks/blob/main/ga360_RCV_14_332_472_Project_1_Question_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classify Dog vs Cat Kaggle dataset with two different networks** Fine-tune a pre-trained network for the dog vs. cat classification problem. Report the confusion matrix, the accuracy, the f-score, precision and recall of your classifier. Write a paragraph describing your results and methods.

In [ ]:
!pip install torch~=2.1.0 torchvision pytorch-lightning
!pip install validators matplotlib
!pip install -U 'torch_xla>=2.1'
!pip install kaggle

In [ ]:
import torch
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
import pytorch_lightning as pl
from sklearn.metrics import classification_report
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from pytorch_lightning.callbacks import ModelCheckpoint
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
torch.__version__
# Define data transformations
transform = ToTensor()
train_folder = '/content/drive/My Drive/path_to_train_data_folder'
test_folder = '/content/drive/MyDrive/Colab Notebooks/Project 1/dogs vs cats Kaggle Data/test1'

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize your images to the desired size
    transforms.ToTensor(),         # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the data
])

train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet model for MNIST classification
class ResNetMNIST(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = resnet18(num_classes=10)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.005)

# Create an instance of the ResNetMNIST model
model = ResNetMNIST()

# Create a trainer for training the model
trainer = pl.Trainer(
    accelerator="cpu",
    max_epochs=1
)

# Train the model
trainer.fit(model, train_loader)

# Save the trained model
trainer.save_checkpoint("resnet18_mnist.pt")

# Define a function for making predictions
def get_prediction(x, model):
    model.freeze()  # Prepare the model for predicting
    probabilities = torch.softmax(model(x), dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    return predicted_class, probabilities

# Load the trained model for inference
inference_model = ResNetMNIST.load_from_checkpoint("resnet18_mnist.pt", map_location="cpu")

true_labels, predicted_labels = [], []

# Perform inference on the test data
for batch in tqdm(test_loader, total=len(test_loader)):
    x, y = batch
    true_labels.extend(y)
    preds, _ = get_prediction(x, inference_model)
    predicted_labels.extend(preds.cpu())

# Print the classification report
print(classification_report(true_labels, predicted_labels, digits=3))

# Create a confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Calculate accuracy, F1-score, precision, and recall
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

print("Confusion Matrix:")
print(conf_matrix)
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")